# AUC-ROC

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report

# Objectives

- Calculate and interpret probability estimates
- Adjust the threshold of a logistic regression model
- Visualize, calculate and interpret the AUC-ROC metric

# Motivation

Now that we've learned how to evaluate a classification model's predictions, let's dig deeper to see how else we might evaluate our models and how we can use that information to improve them.

# Scenario: Identifying Fraudulent Credit Card Transactions

Let's revisit the fraudulent credit card transaction scenario from [this Kaggle dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In [2]:
# Load data
credit_data = pd.read_csv('credit_fraud_small.csv')

# Separate data into feature and target DataFrames
X = credit_data.drop('Class', axis = 1)
y = credit_data['Class']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25,
                                                   random_state=1)
# Scale the data for modeling
cred_scaler = StandardScaler()
cred_scaler.fit(X_train)
X_train_sc = cred_scaler.transform(X_train)
X_test_sc = cred_scaler.transform(X_test)

# Train a logistic regresssion model with the train data
cred_model = LogisticRegression(random_state=42)
cred_model.fit(X_train_sc, y_train)

LogisticRegression(random_state=42)

## Predicting Labels

Let's look at some predictions from our example.

In [3]:
y_pred = cred_model.predict(X_test_sc)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

When we run the `.predict()` method, `sklearn` gives us the predicted values for each transaction in our test set: 0 if predicting "not fraudulent", 1 if predicting "fraudulent"

## Probability Estimates

If you remember how the logistic regression model works, though, it doesn't actually generate predicted values of 0 or 1. It creates an S-shaped curve to approximate the data, estimating the _probability_ that they belong to the target class. This probability takes a value _between_ 0 and 1.

![](https://www.graphpad.com/guides/prism/latest/curve-fitting/images/hmfile_hash_38a8acae.png)

Source: [GraphPad](https://www.graphpad.com/guides/prism/latest/curve-fitting/reg_simple_logistic_and_linear_difference.htm)

We can get these estimated probabilities using the `.predict_proba()` method. Each element gives two probabilities: the estimate probability of being in the 0 class (not fraud) and the 1 class (fraud)

In [4]:
y_prob = cred_model.predict_proba(X_test_sc)
y_prob[:10]

array([[9.99949036e-01, 5.09641701e-05],
       [9.99976279e-01, 2.37213738e-05],
       [9.99998007e-01, 1.99332418e-06],
       [9.99994131e-01, 5.86929843e-06],
       [9.99991505e-01, 8.49504830e-06],
       [9.99990734e-01, 9.26611964e-06],
       [9.99994104e-01, 5.89620203e-06],
       [9.99857038e-01, 1.42961890e-04],
       [9.99991397e-01, 8.60331531e-06],
       [9.99991424e-01, 8.57631437e-06]])

## Thresholds

How did we get those 0 and 1 label predictions, when the model only calculates probabilities between 0 and 1? 

Now the default behavior is simply to take the larger of these values as the "real" prediction. Since $0.99995 > 0.00005$, we'll understand the model to be predicting this point to belong to class "0" (or the positive class). An equivalent way of understanding the default behavior is that we:

- round the predicted numbers up to 1 if they are at least as large as 0.5; and
- round them down to 0 if they are less than 0.5.

Since the probabilities must sum to 1, there will never be any problem with this algorithm. We refer to this value of 0.5 as the **threshold**.

But we don't have to do things this way. Suppose we're building a model that predicts the presence of prostate cancer from X-ray scans of prostates. And suppose we get a pair of probabilities for some particular scan that look like this:

- pred_neg: 0.52, pred_pos: 0.48

Because false negatives (cancerous prostates mislabeled) are *much* more costly than false positives (non-cancerous prostates mislabeled), we may well want to **adjust our threshold** of classification. We might want to have our model predict "positive" if the corresponding probability is, say, as low as 0.4, or maybe even as low as 0.1. (Speaking for myself, if there was even a 10% chance that my prostate was cancerous, I think I'd probably want to know about it.) This can have the effect of improving some of our evaluation metrics while hurting others.

## True & False Positive Rates

Before we proceed, let's discuss two new metrics: **True Positive Rate (TPR)** and **False Positive Rate (FPR)**. Let's review the confusion matrix

In [6]:
cm = confusion_matrix(y_test, cred_model.predict(X_test_sc))

In [7]:
cm

array([[2493,    0],
       [   3,    4]])

Defining true/false positives/negatives:

In [8]:
tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]

TPR & FPR

In [ ]:
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

## Adjusting the threshold

The true- and false-positive rates will change if we make adjustments to the threshold. In fact, in the present case that was the whole point of making the adjustment: We want to minimize our false negatives.

So this is how the plot of these rates takes shape.

Let's build a function that will take in our data, together with a threshold setting, and return the corresponding true- and false-positive rates.

## The Area Under the Curve of the Receiver Operating Characteristic

### AUC-ROC

The Receiver Operating Characteristic curve plots the true-positive rate vs. the false-positive rate. Let's define these now:

In [ ]:
def classify_rates(X_train, X_test, y_train, y_test, thresh):
    logreg = LogisticRegression(solver='liblinear')
    logreg.fit(X_train, y_train)
    y_hat_probs = logreg.predict_proba(X_test)
    y_hat = []
    for val in y_hat_probs:                                 # Each element in y_hat_probs is an array.
        if val[0] < thresh:                                 # We'll set our own threshold for classifying
            y_hat.append(1)                                 # a test point as positive! The lower my threshold,
        else:                                               # the fewer predicted positives I'll have. For the
            y_hat.append(0)                                 # cancer example, I'd want to set a *high* threshold.
    cm = confusion_matrix(y_test, y_hat)
    tp, tn, fp, fn = cm[1][1], cm[0][0], cm[0][1], cm[1][0]
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return tpr, fpr, f'tpr:{round(tpr, 3)}, fpr:{round(fpr, 3)}'

True- and false-positive rates for various thresholds:

In [ ]:
for x in np.linspace(0, 1, 11):
    print(f'Rates at threshold = {round(x, 1)}: ' + classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[2])

As my threshold goes up, I'll have more positive predictions, which means I'll have both more true positives and more false positives.

Note:

- I can artificially increase my true-positive rate to 1 by setting my threshold to 1, but at that point my false-positive rate is also 1! I'll have no true negatives and no false negatives. This will arise naturally if my training data has **very few (actual) negatives**. This was the problem in Lottery Scenario 1.
- I can artificially reduce my false-positive rate to 0 by setting my threshold to 0, but at  that point my true-positive rate is also 0! I'll have no true positives and no false positives. This will arise naturally if my training data has **very few (actual)  positives**. This was the problem in Lottery Scenario 2.

#### Area Under the Curve

The ROC curve will be a plot of tpr (on the y-axis) vs. fpr (on the x-axis). There will always be a point at (0, 0) and another at (1, 1). The question is what happens in the middle. Since we want our y-values to be as high as possible for any particular x-value, a natural metric is to calculate the **area under the curve**. The larger the area, the better the classifier. The maximum possible area is the area of the whole box between 0 and 1 on both axes, so that's a **maximum area of 1**.

What's the minimum? Well that depends on the ratios of (actual) positive and negatives in my data, in much the way that a baseline accuracy score does.

Remember: If my test data comprises 90% positives and only 10% negatives, then a simple classifier that always predicts "positive" will be 90% accurate! And so that would be the baseline level for a classifier on that data.

If we have equal numbers of positives and negatives, then we can set an **abolute minimum area of 0.5**. That's the "curve" we'd get by plotting a straight diagonal line from (0, 0) to (1, 1).

Why? The area under the curve really represents the test's ability to **discriminate** positives from negatives. Suppose I randomly took several pairs of points, one positive and one negative, and checked my test's predictions. The area under the curve represents a threshold-independent measure of how often my test would get the two predictions correct.

#### Plotting the Curve

Let's plot our own ROC curve. We'll create an array of different thresholds and use our `classify_rates()` function to get the true- and false-positive rates for each threshold.

One way of choosing a threshold **independently of business concerns** is to select the point on the curve that is furthest from (1, 0), the "worse-case" point where our true-positive rate is 0 and our false-positive rate is 1. So let's find that point as well:

In [ ]:
tprs = []
fprs = []
diffs = []
for x in np.linspace(0, 1, 101):
    fprs.append(classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[1])
    tprs.append(classify_rates(X_train_sc, X_test_sc, y_train, y_test, x)[0])
    diffs.append(np.sqrt(tprs[-1]**2 + (1-fprs[-1])**2))
    
max_dist = diffs.index(np.max(diffs))
print(f"""With a threshold of {(max_dist - 1) / 100}: \n"""
      f"""\tYou\'ll have a True Positive Rate of {round(tprs[max_dist], 3)} \n"""
      f"""\tand a False Positive Rate of {round(fprs[max_dist], 3)}""")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fprs[:max_dist], tprs[:max_dist], 'r.')
ax.plot(fprs[max_dist], tprs[max_dist], 'ko', ms=10)
ax.plot(fprs[max_dist + 1:], tprs[max_dist + 1:], 'r.')
ax.plot(fprs, fprs, '.');

Scikit-Learn's `roc_auc_score()` function will compute the area under the curve for us:

In [ ]:
round(roc_auc_score(y_test, logreg.predict(X_test_sc)), 4)

Let's compare our curve with scikit-learn's:

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict(X_test_sc))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fpr, tpr);

Scikit-Learn only shows us the optimal threshold, but it appears to be very similar to ours.

# Scenario: Breast Cancer Prediction

Let's evaulate a model using scikit-learn's breast cancer dataset:

In [127]:
# Load the data
preds, target = load_breast_cancer(return_X_y=True)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(preds, target,
                                                   random_state=42)

# Scale the data
bc_scaler = StandardScaler()
bc_scaler.fit(X_train)
X_train_sc = bc_scaler.transform(X_train)
X_test_sc = bc_scaler.transform(X_test)

# Run the model
bc_model = LogisticRegression(solver='lbfgs', max_iter=10000,
                           random_state=42)
bc_model.fit(X_train_sc, y_train)

LogisticRegression(max_iter=10000, random_state=42)

## Task

Calculate the following for this model:

- Confusion Matrix
- Accuracy
- Precision
- Recall
- F1 Score

Discuss: Which one would you choose to evaluate the model for use as a diagnostic tool to detect breast cancer? Why?

In [128]:
# Your work here

In [129]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
y_pred = bc_model.predict(X_test_sc)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[53  1]
 [ 2 87]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        54
           1       0.99      0.98      0.98        89

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143



In [118]:
credit_data = pd.read_csv('credit_fraud_small.csv')

# Separate data into feature and target DataFrames
X = credit_data.drop('Class', axis = 1)
y = credit_data['Class']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25,
                                                   random_state=1)
# Scale the data for modeling
cred_scaler = StandardScaler()
cred_scaler.fit(X_train)
X_train_sc = cred_scaler.transform(X_train)
X_test_sc = cred_scaler.transform(X_test)

# Train a logistic regresssion model with the train data
cred_model = LogisticRegression(random_state=42)
cred_model.fit(X_train_sc, y_train)

LogisticRegression(random_state=42)

# Oversampling

One of the most effective strategies to handle these cases is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.org/stable/over_sampling.html).

# Loss Functions

Another more "natural" way of measuring the quality of a classifier is just to look at the loss function, which will often be the **log loss**. In multiclass problems, we use **cross-entropy loss**. 

In [1]:
log_loss(y_test, y_pred)

NameError: name 'log_loss' is not defined

While such loss values are difficult to interpret on their own, they are very useful for comparing models. Models with lower loss values generate probability estimates that are closer to the true values, and thus are likely to perform better on many metrics.

# Level Up: Log Loss by Hand

Log loss is generally calculated as an average per data point, and is computed as follows:

$L(y, \hat{y}) = -\frac{1}{N}\sum^N_{i=1}[y_i\ln(\hat{y_i}) + (1-y_i)\ln(1-\hat{y_i})]$,

where $y$ is the vector of true values and $\hat{y}$ is the vector of probabilities that the point in question has a correct label of 1.

- Suppose, for a given data point, that the correct prediction of the label is **0**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(1-\hat{y_i})$. So, the closer the prediction for that point is to 0, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 1, the closer the contribution will be to $-\ln(0)=\infty$.

- Suppose, on the other hand, that the correct prediction is **1**. In that case, the contribution from that point to the sum in the loss function defined above will be $-\ln(\hat{y_i})$. So, the closer the prediction for that point is to 1, the closer the contribution to the sum will be to $-\ln(1)=0$. But as the prediction gets closer to 0, the closer the contribution will be to $-\ln(0)=\infty$.

In [124]:
compare = list(zip(y_test, y_pred))

In [125]:
compare[:5]

[(0, 1), (0, 0), (0, 1), (0, 1), (0, 0)]

In [126]:
calc = [-(yi * np.log(yi_hat[1]) + (1 - yi) * np.log(yi_hat[0])) for (yi, yi_hat) in compare]
calc[:5]

IndexError: invalid index to scalar variable.

In [12]:
np.mean(calc)

0.3322228531070542